In [ ]:
# 1. TPU 관련 라이브러리 및 YOLO 설치
# (Colab TPU 런타임 기준, 이미 설치되어 있을 수 있으나 호환성 확보를 위해 실행)
!pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
!pip install ultralytics roboflow pyyaml

# 2. XLA 모듈 임포트 테스트
import torch
import torch_xla
import torch_xla.core.xla_model as xm

print(f"✅ PyTorch Version: {torch.__version__}")
print(f"✅ TPU Device Available: {xm.xla_device()}")

In [ ]:
import sys
import gc
import torch
import torch.nn as nn
import yaml
import torch_xla.core.xla_model as xm  # XLA 모듈 필수

from ultralytics import YOLO
from ultralytics.nn import tasks, modules
from roboflow import Roboflow

# ======================================================================================
# 1. CBAM 모듈 정의 (PyTorch 코드는 TPU에서도 그대로 동작함)
# ======================================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


class CBAM_Universal(nn.Module):
    def __init__(self, *args, **kwargs):
        super(CBAM_Universal, self).__init__()
        c1 = args[0]
        k = 7
        print(f"✅ CBAM 생성됨 | 적용 채널: {c1}")
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(k)

    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out

# ======================================================================================
# 2. Ultralytics 내부에 모듈 등록
# ======================================================================================
target_name = 'CBAM_Universal'

for module_list in [tasks, modules]:
    setattr(module_list, target_name, CBAM_Universal)

if 'ultralytics.nn.modules' in sys.modules:
    setattr(sys.modules['ultralytics.nn.modules'], target_name, CBAM_Universal)

print(f"✅ [준비 완료] 모듈 '{target_name}' 시스템 등록됨.")

In [ ]:
cbam_yaml = {
    'nc': 3,
    'scales': {'m': [0.67, 0.75, 768]},
    'backbone': [
        [-1, 1, 'Conv', [64, 3, 2]],
        [-1, 1, 'Conv', [128, 3, 2]],
        [-1, 3, 'C2f', [128, True]],
        [-1, 1, 'Conv', [256, 3, 2]],
        [-1, 6, 'C2f', [256, True]],
        [-1, 1, target_name, [192]],
        [-1, 1, 'Conv', [512, 3, 2]],
        [-1, 6, 'C2f', [512, True]],
        [-1, 1, target_name, [384]],
        [-1, 1, 'Conv', [1024, 3, 2]],
        [-1, 3, 'C2f', [1024, True]],
        [-1, 1, target_name, [576]],
        [-1, 1, 'SPPF', [1024, 5]]
    ],
    'head': [
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 8], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [512]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 5], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [256]],
        [-1, 1, 'Conv', [256, 3, 2]],
        [[-1, 15], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [512]],
        [-1, 1, 'Conv', [512, 3, 2]],
        [[-1, 11], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [1024]],
        [[18, 21, 24], 1, 'Detect', ['nc']]
    ]
}

yaml_name = 'yolov8m_cbam_real_final.yaml'
with open(yaml_name, 'w') as f:
    yaml.dump(cbam_yaml, f, sort_keys=False)

print(f"✅ [준비 완료] YAML 파일 생성 완료: {yaml_name}")

In [ ]:
RF_API_KEY = "HG9M6YJZpcCUgAQaKO9v"
WORKSPACE = "arakon"
PROJECT = "detection-base-hqaeg"
VERSION = 7
FORMAT = "yolov8"

rf = Roboflow(api_key=RF_API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
version = project.version(VERSION)
dataset = version.download(FORMAT)

data_path = f"{dataset.location}/data.yaml"
print("✅ Roboflow 데이터셋 위치:", dataset.location)

In [ ]:
def train_stage(
    yaml_path: str,
    data_yaml: str,
    imgsz: int,
    batch: int,
    epochs: int,
    patience: int,
    run_name: str,
    device=None,  # TPU Device 객체를 받기 위해 None으로 변경
    workers: int = 8, # TPU는 데이터 로딩 속도가 중요하므로 워커 수 상향
    cache = True,    # TPU IO 병목 방지를 위해 가능하면 True 권장
    lr0: float = 0.001,
    lrf: float = 0.01,
    cos_lr: bool = True,
    optimizer: str = "AdamW",
):
    print(f"\n🚀 TPU 학습 시작 | imgsz={imgsz}, batch={batch}, epochs={epochs}, run={run_name}")
    print(f"ℹ️  Device Info: {device}")

    # 모델 로드
    model = YOLO(yaml_path).load("yolov8m.pt")

    # 학습 실행
    model.train(
        data=data_yaml,
        epochs=epochs,
        patience=patience,
        imgsz=imgsz,
        batch=batch,
        optimizer=optimizer,
        lr0=lr0,
        lrf=lrf,
        cos_lr=cos_lr,
        cache=cache,
        device=device,  # [핵심] 여기에 xm.xla_device()가 들어감
        workers=workers,
        name=run_name,
        exist_ok=True,
        deterministic=False,
        plots=True
    )

    # 메모리 정리
    del model
    gc.collect()
    # TPU 메모리 정리는 자동이지만 명시적으로 호출
    xm.mark_step() 
    
    print(f"✅ 학습 종료 | {run_name}")

In [ ]:
# TPU Device 객체 획득
tpu_device = xm.xla_device()

# 학습 실행
train_stage(
    yaml_path=yaml_name,
    data_yaml=data_path,
    imgsz=800,
    batch=16,     # TPU 효율을 위해 8 -> 16으로 상향 조정 (메모리 문제시 8로 롤백)
    epochs=100,
    patience=30,
    run_name="tank_yolo_cbam_800_tpu",
    device=tpu_device, # [중요] TPU 디바이스 전달
    workers=8,    # 데이터 로더 워커 수
    cache=True    # RAM 여유가 된다면 True 권장
)

print("\n🎉 TPU 학습 완료! 🚀")

In [ ]:
final_weights_path = "/content/runs/detect/tank_yolo_cbam_800_tpu/weights/best.pt"
# 참고: Colab 기본 경로는 /content/runs/... 일 확률이 높음. workspace 경로 확인 필요.

print(f"🚀 검증 시작: {final_weights_path}")

try:
    best_model = YOLO(final_weights_path)
    
    # 검증 실행
    metrics = best_model.val(
        data=data_path,
        split='test',
        imgsz=800,
        device=tpu_device # 검증도 TPU 사용
    )

    print(f"\n✅ [Test Set 결과] mAP50: {metrics.box.map50:.4f}")
    print(f"✅ [Test Set 결과] mAP50-95: {metrics.box.map:.4f}")

except Exception as e:
    print(f"⚠️ 모델 로드 또는 검증 중 오류 발생 (경로 확인 필요): {e}")